In [237]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [238]:
df = pd.read_csv('cleaned_data2.csv')

In [239]:
df['date']=pd.to_datetime(df['date'], format='%Y-%m-%d')

In [240]:
df.head()

,record,card,date,merch#,merchdes,merchstate,merchzip,transtype,amount,fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706,P,178.49,0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118,P,3.62,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0


## Combine related variables

In [241]:
df['card-merchant'] = df['card'].astype(str) + '-' + df['merch#']
df['card-zip'] = df['card'].astype(str) + '-' + df['merchzip'].astype(str)
df['card-state'] = df['card'].astype(str) + '-' + df['merchstate']

In [242]:
attributes = list(df.columns)
for col in ['record', 'date', 'merchdes', 'merchstate', 'merchzip', 'transtype','amount','fraud']:
    attributes.remove(col)
print(len(attributes))

5


In [244]:
finalDF = df[['record','date','amount']]

for val in attributes:
    df1 = df[['record','date', 'amount',val]].copy()
    df2 = df1.copy()
    lags = [0, 1, 3, 7, 14, 30]
    
    for lag in lags:
        temp_name = 'date_{}'.format(lag)
        df2[temp_name] = df2['date'] + dt.timedelta(lag)

    dff = df1.merge(df2, on=val)

    tmp = dff[(dff['record_x']>dff['record_y'])][['record_x','date_y']].\
          groupby('record_x').last()['date_y'].reset_index()
    tmp.columns = ['record','last_seen']
    df1 = df1.merge(tmp, how='left', on='record')
    df1['#d_since'] = df1['date'] - df1['last_seen']
    df1['#d_since'] = df1['#d_since'].dt.days
    df1['#d_since_start'] = (df1['date']-df1['date'].min()).dt.days
    df1['#d_since'] = df1['#d_since'].fillna(df1['#d_since_start'])
    
    df1.drop(columns=['last_seen','#d_since_start'], inplace=True)

    for lag in lags:
        temp = dff[(dff['record_x']>dff['record_y']) & (dff['date_x']<=dff['date_{}'.format(lag)])]
        tempCount = temp[['record_x','record_y']].groupby('record_x').count().reset_index()
        temp_name = 'ps_{}d_count'.format(lag)
        tempCount.columns = ['record', temp_name]
        df1 = df1.merge(tempCount, left_on='record',right_on='record',how='left')
        df1[temp_name]=df1[temp_name].fillna(0)
    
    ###amount variables###
    
    for lag in lags:
        ##Average##
        temp = dff[(dff['record_x']>dff['record_y']) & (dff['date_x']<=dff['date_{}'.format(lag)])]
        tempAverage = temp[['record_x','amount_y']].groupby('record_x').mean().reset_index()
        temp_name = 'ps_{}d_average'.format(lag)
        tempAverage.columns = ['record', temp_name]
        df1 = df1.merge(tempAverage, left_on='record',right_on='record',how='left')
        df1[temp_name]=df1[temp_name].fillna(0.00000001)
        
                ##Maximum##
        tempMaximum = temp[['record_x','amount_y']].groupby('record_x').max().reset_index()
        temp_name = 'ps_{}d_maximum'.format(lag)
        tempMaximum.columns = ['record', temp_name]
        df1 = df1.merge(tempMaximum, left_on='record',right_on='record',how='left')
        df1[temp_name]=df1[temp_name].fillna(0.00000001)
        
                 ##Median##
        tempMedian = temp[['record_x','amount_y']].groupby('record_x').median().reset_index()
        temp_name = 'ps_{}d_median'.format(lag)
        tempMedian.columns = ['record', temp_name]
        df1 = df1.merge(tempMedian, left_on='record',right_on='record',how='left')
        df1[temp_name]=df1[temp_name].fillna(0.00000001)
        
                        ##Total##
        tempTotal = temp[['record_x','amount_y']].groupby('record_x').sum().reset_index()
        temp_name = 'ps_{}d_total'.format(lag)
        tempTotal.columns = ['record', temp_name]
        df1 = df1.merge(tempTotal, left_on='record',right_on='record',how='left')
        df1[temp_name]=df1[temp_name].fillna(0.00000001)
        
                        ##actual/average##
        temp_name = 'ps_{}d_ActualAverage'.format(lag)
        tempDenominator_avg = 'ps_{}d_average'.format(lag)
        df1[temp_name] = df1['amount']/df1[tempDenominator_avg]
        
                        ##actual/maximum##
        temp_name = 'ps_{}d_ActualMaximum'.format(lag)
        tempDenominator_max = 'ps_{}d_maximum'.format(lag)
        df1[temp_name] = df1['amount']/df1[tempDenominator_max]
        
                        ##actual/median##
        temp_name = 'ps_{}d_ActualMedian'.format(lag)
        tempDenominator_median = 'ps_{}d_median'.format(lag)
        
        df1[temp_name] = df1['amount']/df1[tempDenominator_median]
                                ##actual/total##
        temp_name = 'ps_{}d_ActualTotal'.format(lag)
        tempDenominator_total = 'ps_{}d_total'.format(lag)
        df1[temp_name] = df1['amount']/df1[tempDenominator_total]

    ### Velocity change variables###
    
    #number#
    for i in [0,1]:
        for j in [3, 7, 14, 30]:
            temp_name = 'ps_{}d_ps{}d_number'.format(i,j)
            numerator = 'ps_{}d_count'.format(i)
            denominator = 'ps_{}d_count'.format(j)
            df1[temp_name] = df1[numerator] / (df1[denominator]/j)
            df1[temp_name]=df1[temp_name].fillna(0)
    
    #amount#
    for i in [0,1]:
        for j in [3, 7, 14, 30]:
            temp_name = 'ps_{}d_ps{}d_amount'.format(i,j)
            numerator1 = 'ps_{}d_average'.format(i)
            numerator2 = 'ps_{}d_count'.format(i)
            denominator1 = 'ps_{}d_average'.format(j)
            denominator2 = 'ps_{}d_count'.format(j)
            df1[temp_name] = (df1[numerator1]*df1[numerator2]) / ((df1[denominator1]*df1[denominator2]) /j) 
            df1[temp_name]=df1[temp_name].fillna(0)
    
    df1.drop(columns=['date',val], inplace=True)
    df1 = df1.set_index('record')
    df1 = df1.add_prefix('{}_'.format(val))
    finalDF = finalDF.merge(df1, left_on='record', right_index=True)

In [245]:
finalDF.head()

,record,date,amount,card_amount,card_#d_since,card_ps_0d_count,card_ps_1d_count,card_ps_3d_count,card_ps_7d_count,card_ps_14d_count,...,card-state_ps_1d_ps14d_number,card-state_ps_1d_ps30d_number,card-state_ps_0d_ps3d_amount,card-state_ps_0d_ps7d_amount,card-state_ps_0d_ps14d_amount,card-state_ps_0d_ps30d_amount,card-state_ps_1d_ps3d_amount,card-state_ps_1d_ps7d_amount,card-state_ps_1d_ps14d_amount,card-state_ps_1d_ps30d_amount
0,1,2010-01-01,3.62,3.62,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,2010-01-01,31.42,31.42,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,2010-01-01,178.49,178.49,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,2010-01-01,3.62,3.62,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,2010-01-01,3.62,3.62,0.0,1.0,1.0,1.0,1.0,1.0,...,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0


In [246]:
data=finalDF.copy()

In [247]:
finalDF.shape

(96397, 363)

## Create Benford's Law variable

In [248]:
for i in ['merchdes','merch#','card']:
    data[i]=df[i]

In [249]:
data.head()

,record,date,amount,card_amount,card_#d_since,card_ps_0d_count,card_ps_1d_count,card_ps_3d_count,card_ps_7d_count,card_ps_14d_count,...,card-state_ps_0d_ps7d_amount,card-state_ps_0d_ps14d_amount,card-state_ps_0d_ps30d_amount,card-state_ps_1d_ps3d_amount,card-state_ps_1d_ps7d_amount,card-state_ps_1d_ps14d_amount,card-state_ps_1d_ps30d_amount,merchdes,merch#,card
0,1,2010-01-01,3.62,3.62,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,FEDEX SHP 12/23/09 AB#,5509006296254,5142190439
1,2,2010-01-01,31.42,31.42,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SERVICE MERCHANDISE #81,61003026333,5142183973
2,3,2010-01-01,178.49,178.49,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OFFICE DEPOT #191,4503082993600,5142131721
3,4,2010-01-01,3.62,3.62,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,FEDEX SHP 12/28/09 AB#,5509006296254,5142148452
4,5,2010-01-01,3.62,3.62,0.0,1.0,1.0,1.0,1.0,1.0,...,7.0,14.0,30.0,3.0,7.0,14.0,30.0,FEDEX SHP 12/23/09 AB#,5509006296254,5142190439


In [250]:
df3 = data[['record','amount','merchdes','merch#','card']]

In [251]:
df3['first_digit'] = (df3['amount']*100).astype(str).apply(lambda x:x[0])

/Users/selenexiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [252]:
df3['low']=np.where((df3['first_digit'].str.contains('1|2'))&(df3['merchdes']\
                                                              .str.contains('FEDEX|FEDERAL EXPRESS|FED EX')==False),1,0)
df3['high']=1-df3['low']
df3['high']=np.where(df3['merchdes'].str.contains('FEDEX|FEDERAL EXPRESS|FED EX'),0,df3['high'])

/Users/selenexiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/selenexiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/selenexiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

In [253]:
df3.head()

,record,amount,merchdes,merch#,card,first_digit,low,high
0,1,3.62,FEDEX SHP 12/23/09 AB#,5509006296254,5142190439,3,0,0
1,2,31.42,SERVICE MERCHANDISE #81,61003026333,5142183973,3,0,1
2,3,178.49,OFFICE DEPOT #191,4503082993600,5142131721,1,1,0
3,4,3.62,FEDEX SHP 12/28/09 AB#,5509006296254,5142148452,3,0,0
4,5,3.62,FEDEX SHP 12/23/09 AB#,5509006296254,5142190439,3,0,0


In [254]:
df4=data[['record','merchdes','merch#','card','amount']]

In [255]:
df3.shape

(96397, 8)

In [256]:
df4.shape

(96397, 5)

In [257]:
df4.head()

,record,merchdes,merch#,card,amount
0,1,FEDEX SHP 12/23/09 AB#,5509006296254,5142190439,3.62
1,2,SERVICE MERCHANDISE #81,61003026333,5142183973,31.42
2,3,OFFICE DEPOT #191,4503082993600,5142131721,178.49
3,4,FEDEX SHP 12/28/09 AB#,5509006296254,5142148452,3.62
4,5,FEDEX SHP 12/23/09 AB#,5509006296254,5142190439,3.62


In [258]:
dff_b = df4.merge(df3, on='merch#',how='left')

dff_b.shape

(107898463, 12)

In [259]:
dff_b.head()

,record_x,merchdes_x,merch#,card_x,amount_x,record_y,amount_y,merchdes_y,card_y,first_digit,low,high
0,1,FEDEX SHP 12/23/09 AB#,5509006296254,5142190439,3.62,1,3.62,FEDEX SHP 12/23/09 AB#,5142190439,3,0,0
1,1,FEDEX SHP 12/23/09 AB#,5509006296254,5142190439,3.62,4,3.62,FEDEX SHP 12/28/09 AB#,5142148452,3,0,0
2,1,FEDEX SHP 12/23/09 AB#,5509006296254,5142190439,3.62,5,3.62,FEDEX SHP 12/23/09 AB#,5142190439,3,0,0
3,1,FEDEX SHP 12/23/09 AB#,5509006296254,5142190439,3.62,6,3.67,FEDEX SHP 12/22/09 AB#,5142149874,3,0,0
4,1,FEDEX SHP 12/23/09 AB#,5509006296254,5142190439,3.62,7,3.62,FEDEX SHP 12/28/09 AB#,5142189277,3,0,0


In [260]:
tmp = dff_b[(dff_b['record_x']>=dff_b['record_y'])][['record_x','merchdes_y','first_digit','low','high']] \
        .groupby('record_x')['low','high'].sum().reset_index()

/Users/selenexiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [261]:
tmp.head()

,record_x,low,high
0,1,0,0
1,2,0,1
2,3,1,0
3,4,0,0
4,5,0,0


In [262]:
tmp.columns=['record','n_low','n_high']

In [263]:
tmp['sum']=tmp['n_low']+tmp['n_high']
tmp['n_low']=np.where(tmp['n_low']==0,1,tmp['n_low'])
tmp['n_high']=np.where(tmp['n_high']==0,1,tmp['n_high'])
tmp['R'] = 1.096*tmp['n_low']/tmp['n_high']
tmp['U'] = np.maximum(tmp['R'],1/tmp['R'])

c=3
nmid=15
tmp['U_smooth_merch'] = 1 + (tmp['U'] - 1)/(1 + np.exp(-(tmp['sum']-nmid)/c))

tmp.head()

,record,n_low,n_high,sum,R,U,U_smooth_merch
0,1,1,1,0,1.096,1.096,1.000643
1,2,1,1,1,1.096,1.096,1.000894
2,3,1,1,1,1.096,1.096,1.000894
3,4,1,1,0,1.096,1.096,1.000643
4,5,1,1,0,1.096,1.096,1.000643


In [264]:
tmp.shape

(96397, 7)

In [265]:
tmp=tmp[['record','U_smooth_merch']]
data = data.merge(tmp,on='record',how='left')

In [266]:
# Benford for card#
dff_bc = df4.merge(df3, on='card',how='left')

tmpc = dff_bc[(dff_bc['record_x']>=dff_bc['record_y'])][['record_x','merchdes_y','first_digit','low','high']] \
        .groupby('record_x')['low','high'].sum().reset_index()
tmpc.columns=['record','n_low','n_high']

tmpc['sum']=tmpc['n_low']+tmpc['n_high']
tmpc['n_low']=np.where(tmpc['n_low']==0,1,tmpc['n_low'])
tmpc['n_high']=np.where(tmpc['n_high']==0,1,tmpc['n_high'])
tmpc['R'] = 1.096*tmpc['n_low']/tmpc['n_high']
tmpc['U'] = np.maximum(tmpc['R'],1/tmpc['R'])

c=3
nmid=15
tmpc['U_smooth_card'] = 1 + (tmpc['U'] - 1)/(1 + np.exp(-(tmpc['sum']-nmid)/c))


/Users/selenexiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """


In [267]:
tmpc=tmpc[['record','U_smooth_card']]
tmpc.head()

,record,U_smooth_card
0,1,1.000643
1,2,1.000894
2,3,1.000894
3,4,1.000643
4,5,1.000643


In [268]:
tmpc.shape

(96397, 2)

In [269]:

data = data.merge(tmpc,on='record',how='left')

In [270]:
data.head()

,record,date,amount,card_amount,card_#d_since,card_ps_0d_count,card_ps_1d_count,card_ps_3d_count,card_ps_7d_count,card_ps_14d_count,...,card-state_ps_0d_ps30d_amount,card-state_ps_1d_ps3d_amount,card-state_ps_1d_ps7d_amount,card-state_ps_1d_ps14d_amount,card-state_ps_1d_ps30d_amount,merchdes,merch#,card,U_smooth_merch,U_smooth_card
0,1,2010-01-01,3.62,3.62,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,FEDEX SHP 12/23/09 AB#,5509006296254,5142190439,1.000643,1.000643
1,2,2010-01-01,31.42,31.42,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,SERVICE MERCHANDISE #81,61003026333,5142183973,1.000894,1.000894
2,3,2010-01-01,178.49,178.49,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,OFFICE DEPOT #191,4503082993600,5142131721,1.000894,1.000894
3,4,2010-01-01,3.62,3.62,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,FEDEX SHP 12/28/09 AB#,5509006296254,5142148452,1.000643,1.000643
4,5,2010-01-01,3.62,3.62,0.0,1.0,1.0,1.0,1.0,1.0,...,30.0,3.0,7.0,14.0,30.0,FEDEX SHP 12/23/09 AB#,5509006296254,5142190439,1.000643,1.000643


In [271]:
data['U_smooth_merch'].isnull().sum()

0

In [272]:
data['U_smooth_card'].isnull().sum()

0

## Day of Week risk table

In [273]:
# 0-Monday, 6-Sunday
data['weekday'] = data.date.dt.dayofweek
data = data.merge(df[['record','fraud']],how='left')
data.shape


(96397, 370)

In [274]:
data.head()

,record,date,amount,card_amount,card_#d_since,card_ps_0d_count,card_ps_1d_count,card_ps_3d_count,card_ps_7d_count,card_ps_14d_count,...,card-state_ps_1d_ps7d_amount,card-state_ps_1d_ps14d_amount,card-state_ps_1d_ps30d_amount,merchdes,merch#,card,U_smooth_merch,U_smooth_card,weekday,fraud
0,1,2010-01-01,3.62,3.62,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,FEDEX SHP 12/23/09 AB#,5509006296254,5142190439,1.000643,1.000643,4,0
1,2,2010-01-01,31.42,31.42,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,SERVICE MERCHANDISE #81,61003026333,5142183973,1.000894,1.000894,4,0
2,3,2010-01-01,178.49,178.49,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,OFFICE DEPOT #191,4503082993600,5142131721,1.000894,1.000894,4,0
3,4,2010-01-01,3.62,3.62,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,FEDEX SHP 12/28/09 AB#,5509006296254,5142148452,1.000643,1.000643,4,0
4,5,2010-01-01,3.62,3.62,0.0,1.0,1.0,1.0,1.0,1.0,...,7.0,14.0,30.0,FEDEX SHP 12/23/09 AB#,5509006296254,5142190439,1.000643,1.000643,4,0


In [275]:
# Drop variables
data.drop(['card_amount','merch#_amount','card-merchant_amount','card-zip_amount','card-state_amount'],axis=1,inplace=True)

In [276]:
data.drop(['card-merchant_ps_0d_ps7d_number','card-merchant_ps_0d_ps14d_number',
                      'card-merchant_ps_0d_ps30d_number','card-merchant_ps_1d_ps7d_number',
                      'card-merchant_ps_1d_ps14d_number','card-merchant_ps_1d_ps30d_number',
                      'card-merchant_ps_0d_ps7d_amount','card-merchant_ps_0d_ps14d_amount',
                      'card-merchant_ps_0d_ps30d_amount','card-merchant_ps_1d_ps7d_amount',
                      'card-merchant_ps_1d_ps14d_amount','card-merchant_ps_1d_ps30d_amount',], axis=1,inplace=True)

In [277]:
data.drop(['card-zip_ps_0d_ps7d_number','card-zip_ps_0d_ps14d_number',
                      'card-zip_ps_0d_ps30d_number','card-zip_ps_1d_ps7d_number',
                      'card-zip_ps_1d_ps14d_number','card-zip_ps_1d_ps30d_number',
                      'card-zip_ps_0d_ps7d_amount','card-zip_ps_0d_ps14d_amount',
                      'card-zip_ps_0d_ps30d_amount','card-zip_ps_1d_ps7d_amount',
                      'card-zip_ps_1d_ps14d_amount','card-zip_ps_1d_ps30d_amount'], axis=1,inplace=True)

In [278]:
data.drop(['card-state_ps_0d_ps7d_number','card-state_ps_0d_ps14d_number',
                      'card-state_ps_0d_ps30d_number','card-state_ps_1d_ps7d_number',
                      'card-state_ps_1d_ps14d_number','card-state_ps_1d_ps30d_number',
                      'card-state_ps_0d_ps7d_amount','card-state_ps_0d_ps14d_amount',
                      'card-state_ps_0d_ps30d_amount','card-state_ps_1d_ps7d_amount',
                      'card-state_ps_1d_ps14d_amount','card-state_ps_1d_ps30d_amount'], axis=1,inplace=True)

In [279]:
data.drop(['merchdes','merch#','card'],inplace=True,axis=1)

In [280]:
data.shape

(96397, 326)

In [281]:
data['random']=np.random.random(size=96397)

In [282]:
data.head()

,record,date,amount,card_#d_since,card_ps_0d_count,card_ps_1d_count,card_ps_3d_count,card_ps_7d_count,card_ps_14d_count,card_ps_30d_count,...,card-state_ps_30d_ActualTotal,card-state_ps_0d_ps3d_number,card-state_ps_1d_ps3d_number,card-state_ps_0d_ps3d_amount,card-state_ps_1d_ps3d_amount,U_smooth_merch,U_smooth_card,weekday,fraud,random
0,1,2010-01-01,3.62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.620000e+08,0.0,0.0,0.0,0.0,1.000643,1.000643,4,0,0.203250
1,2,2010-01-01,31.42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.142000e+09,0.0,0.0,0.0,0.0,1.000894,1.000894,4,0,0.408200
2,3,2010-01-01,178.49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.784900e+10,0.0,0.0,0.0,0.0,1.000894,1.000894,4,0,0.577225
3,4,2010-01-01,3.62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.620000e+08,0.0,0.0,0.0,0.0,1.000643,1.000643,4,0,0.175165
4,5,2010-01-01,3.62,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.000000e+00,3.0,3.0,3.0,3.0,1.000643,1.000643,4,0,0.339832


In [283]:
tt = data[data.date<'2010-11-01'].copy()
oot = data[data.date>='2010-11-01'].copy()

In [284]:
c = 4
nmid = 20
y_avg = tt['fraud'].mean()
y_weekday_avg = tt.groupby('weekday')['fraud'].mean()
num_instances_weekday = tt.groupby('weekday').size()
y_weekday_smooth = y_avg + (y_weekday_avg - y_avg)/(1 + np.exp(-(num_instances_weekday-nmid)/c))
tt['weekday_risk'] = tt['weekday'].map(y_weekday_smooth)
oot['weekday_risk'] = oot['weekday'].map(y_weekday_smooth)

In [285]:
tt

,record,date,amount,card_#d_since,card_ps_0d_count,card_ps_1d_count,card_ps_3d_count,card_ps_7d_count,card_ps_14d_count,card_ps_30d_count,...,card-state_ps_0d_ps3d_number,card-state_ps_1d_ps3d_number,card-state_ps_0d_ps3d_amount,card-state_ps_1d_ps3d_amount,U_smooth_merch,U_smooth_card,weekday,fraud,random,weekday_risk
0,1,2010-01-01,3.62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.000643,1.000643,4,0,0.203250,0.025994
1,2,2010-01-01,31.42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.000894,1.000894,4,0,0.408200,0.025994
2,3,2010-01-01,178.49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.000894,1.000894,4,0,0.577225,0.025994
3,4,2010-01-01,3.62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.000643,1.000643,4,0,0.175165,0.025994
4,5,2010-01-01,3.62,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,3.0,3.0,3.0,3.0,1.000643,1.000643,4,0,0.339832,0.025994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83965,84295,2010-10-31,299.77,23.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,1.142441,2.007544,6,0,0.085250,0.009630
83966,84296,2010-10-31,609.34,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,3.0,3.0,3.0,3.0,1.070212,2.298603,6,0,0.524486,0.009630
83967,84297,2010-10-31,235.00,3.0,0.0,0.0,1.0,1.0,2.0,3.0,...,0.0,0.0,0.0,0.0,1.001244,1.847674,6,0,0.098483,0.009630
83968,84298,2010-10-31,600.00,68.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.000894,1.357196,6,0,0.457108,0.009630


In [286]:
tt.to_csv('variables_traintest2-selene.csv', index=False)
oot.to_csv('variables_oot2-selene.csv', index=False)